<a href="https://colab.research.google.com/github/sierra-raby/HRRR-smoke/blob/main/OS_grib2_to_tif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This script processes raw grib files from NOAA's HRRR-Smoke experimental dataset for smoke exposure studies
# First it unzips tar files and finds missing data
# Next it creates daily average files, outputting to tiff
# Finally it uploads to Google Earth Engine (via Google Cloud Storage) for analysis

# Setup - packages
from osgeo import gdal
from osgeo.gdalnumeric import *
from osgeo.gdalconst import *

In [ ]:
# Setup - drive connection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# unzipping tar.gz files
import tarfile
tar = tarfile.open('/content/drive/MyDrive/for_OS/201910.tar.gz')
tar.extractall('/content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910')

In [ ]:
# count number of nonzero files per initialization to find missing initializaitons
import os
directory = "/content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910" # change to select month
# iterate over initalization folders in month directory
for init_folder in os.listdir(directory):
    initPath = os.path.join(directory, init_folder)
    fileSizes = []
    #print(initPath)
    basename = os.path.basename(initPath)
    print(basename)
    # continue to postprd folder
    for postprd in os.listdir(initPath): 
          postprdPath = os.path.join(initPath, postprd)
          # continue to grib file and call size
          for grib in os.listdir(postprdPath):
            file_size = os.path.getsize(postprdPath+'/'+grib) # in bytes
            fileSizes.append(file_size)
    # count number of 0 b files
    numZeros = fileSizes.count(0)
    # subtract number of 0 files from total number of files
    numNonZero = len(fileSizes)-numZeros
    print(str(numNonZero)+" files.")

2019102512
46 files.
2019100106
49 files.
2019100112
49 files.
2019100118
26 files.
2019100200
49 files.
2019100212
39 files.
2019100300
49 files.
2019100306
49 files.
2019100312
49 files.
2019100318
49 files.
2019100400
49 files.
2019100406
49 files.
2019100412
26 files.
2019100418
49 files.
2019100500
49 files.
2019100506
49 files.
2019100512
49 files.
2019100518
49 files.
2019100600
49 files.
2019100606
3 files.
2019100612
20 files.
2019100618
38 files.
2019100700
6 files.
2019100712
21 files.
2019100718
49 files.
2019100800
49 files.
2019100912
49 files.
2019100918
49 files.
2019101000
46 files.
2019101018
49 files.
2019101100
49 files.
2019101106
49 files.
2019101112
49 files.
2019101118
49 files.
2019101200
49 files.
2019101206
49 files.
2019101212
26 files.
2019101218
26 files.
2019101300
49 files.
2019101306
49 files.
2019101312
49 files.
2019101412
49 files.
2019101418
25 files.
2019101512
31 files.
2019101518
49 files.
2019101600
49 files.
2019101606
18 files.
2019101718
49 f

In [ ]:
# Averaging and output to tif
# new code generalizable to 00 or 00/12 initialization files
from string import ascii_uppercase as letter
import os

#  main
if __name__ == "__main__":

    gdal_calc = '/content/drive/MyDrive/NOAA/grib2_files_test/gdal_calc.py'
    command = ['python', gdal_calc]

    # Edit the five variables below to change the date, # files, or how many are initialized at midnight -vs- noon
    year = '2019'
    month = '10'
    day = '21'
    totalFiles = 24
    nfile_init_at_midnight = 24 # keep as 12 except for days with all 12 files missing only then use 24

    dd = year + month + day
    smoke_path1 = "/content/drive/MyDrive/HRRRsmoke_OS/grib2_files/" + year + month + "/"
    #smoke_path1 = "/content/drive/MyDrive/NOAA/grib2_files/" + year + month + "/"
    smoke_path2 = "/postprd/nearsfc_smoke_"
    smoke_path3 = ".grib2"

    ifile = 0;
    strStart = '00'
    for i in range(totalFiles):
        command.append('-' + letter[i])
        ordinal = str(ifile)
        if ifile<10:
            ordinal = '0' + ordinal
        command.append(smoke_path1 + dd + strStart + smoke_path2 + ordinal + smoke_path3)
        ifile = ifile + 1
        if ifile >= nfile_init_at_midnight:
            ifile = 0
            strStart = '12'

    output = "/content/drive/MyDrive/HRRRsmoke_OS/averaged2019/" + dd + ".tiff"
    #output = "/content/drive/MyDrive/NOAA/grib2_files/averaged3/" + dd + ".tiff"
    command.append("--outfile=" + output)
    letrList = '+'.join(letter[:totalFiles])
    command.append('--calc="(' + letrList + ')/' + str(totalFiles) + '"')
    # command.append('--calc="(A+B+C+D+E+F+G+H+I+J+K+L+M+N+O+P+Q+R+S+T+U+V+W+X)/24"')
    theCommand = ' '.join(command)
    print('command=[' + theCommand + ']')
    os.system(theCommand)


command=[python /content/drive/MyDrive/NOAA/grib2_files_test/gdal_calc.py -A /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_00.grib2 -B /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_01.grib2 -C /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_02.grib2 -D /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_03.grib2 -E /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_04.grib2 -F /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_05.grib2 -G /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_06.grib2 -H /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_07.grib2 -I /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_08.grib2 -J /content/drive/MyDrive/

In [ ]:
# Averaging and output to tif
# new code generalizable to 00 or 00/12 initialization files
from string import ascii_uppercase as letter
import os

#  main
if __name__ == "__main__":

    gdal_calc = '/content/drive/MyDrive/NOAA/grib2_files_test/gdal_calc.py'
    command = ['python', gdal_calc]

    # Edit the five variables below to change the date, # files, or how many are initialized at midnight -vs- noon
    year = '2019'
    month = '10'
    day = '20'
    totalFiles = 24
    nfile_init_at_midnight = 24 # keep as 12 except for days with all 12 files missing only then use 24

    dd = year + month + day
    smoke_path1 = "/content/drive/MyDrive/HRRRsmoke_OS/grib2_files/" + year + month + "/"
    #smoke_path1 = "/content/drive/MyDrive/NOAA/grib2_files/" + year + month + "/"
    smoke_path2 = "/postprd/nearsfc_smoke_"
    smoke_path3 = ".grib2"

    ifile = 0;
    strStart = '00'
    for i in range(totalFiles):
        command.append('-' + letter[i])
        ordinal = str(ifile)
        if ifile<10:
            ordinal = '0' + ordinal
        command.append(smoke_path1 + dd + strStart + smoke_path2 + ordinal + smoke_path3)
        ifile = ifile + 1
        if ifile >= nfile_init_at_midnight:
            ifile = 0
            strStart = '12'

    output = "/content/drive/MyDrive/HRRRsmoke_OS/averaged2019/" + dd + ".tiff"
    #output = "/content/drive/MyDrive/NOAA/grib2_files/averaged3/" + dd + ".tiff"
    command.append("--outfile=" + output)
    letrList = '+'.join(letter[:totalFiles])
    command.append('--calc="(' + letrList + ')/' + str(totalFiles) + '"')
    # command.append('--calc="(A+B+C+D+E+F+G+H+I+J+K+L+M+N+O+P+Q+R+S+T+U+V+W+X)/24"')
    theCommand = ' '.join(command)
    print('command=[' + theCommand + ']')
    os.system(theCommand)


command=[python /content/drive/MyDrive/NOAA/grib2_files_test/gdal_calc.py -A /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_00.grib2 -B /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_01.grib2 -C /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_02.grib2 -D /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_03.grib2 -E /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_04.grib2 -F /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_05.grib2 -G /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_06.grib2 -H /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_07.grib2 -I /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102000/postprd/nearsfc_smoke_08.grib2 -J /content/drive/MyDrive/

In [ ]:
# special case 1 - no 00
import os
import os
! python /content/drive/MyDrive/NOAA/grib2_files_test/gdal_calc.py \
    -A  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_12.grib2 \
    -B  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_13.grib2 \
    -C  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_14.grib2 \
    -D  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_15.grib2 \
    -E  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_16.grib2 \
    -F  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_17.grib2 \
    -G  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_18.grib2 \
    -H  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_19.grib2 \
    -I  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_20.grib2 \
    -J  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_21.grib2 \
    -K  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_22.grib2 \
    -L  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101412/postprd/nearsfc_smoke_23.grib2 \
    -M  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_00.grib2 \
    -N  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_01.grib2 \
    -O  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_02.grib2 \
    -P  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_03.grib2 \
    -Q  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_04.grib2 \
    -R  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_05.grib2 \
    -S  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_06.grib2 \
    -T  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_07.grib2 \
    -U  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_08.grib2 \
    -V  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_09.grib2 \
    -W  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_10.grib2 \
    -X  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019101512/postprd/nearsfc_smoke_11.grib2 \
    --outfile=/content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20191015.tiff  \
    --calc="(A+B+C+D+E+F+G+H+I+J+K+L+M+N+O+P+Q+R+S+T+U+V+W+X)/24"

In [ ]:
# special case 2 - no 00, previous day has no 12
import os
! python /content/drive/MyDrive/NOAA/grib2_files_test/gdal_calc.py \
    -A  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_24.grib2 \
    -B  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_25.grib2 \
    -C  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_26.grib2 \
    -D  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_27.grib2 \
    -E  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_28.grib2 \
    -F  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_29.grib2 \
    -G  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_30.grib2 \
    -H  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_31.grib2 \
    -I  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_32.grib2 \
    -J  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_33.grib2 \
    -K  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_34.grib2 \
    -L  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100800/postprd/nearsfc_smoke_35.grib2 \
    -M  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_00.grib2 \
    -N  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_01.grib2 \
    -O  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_02.grib2 \
    -P  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_03.grib2 \
    -Q  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_04.grib2 \
    -R  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_05.grib2 \
    -S  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_06.grib2 \
    -T  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_07.grib2 \
    -U  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_08.grib2 \
    -V  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_09.grib2 \
    -W  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_10.grib2 \
    -X  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019100912/postprd/nearsfc_smoke_11.grib2 \
    --outfile=/content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20191009.tiff  \
    --calc="(A+B+C+D+E+F+G+H+I+J+K+L+M+N+O+P+Q+R+S+T+U+V+W+X)/24"

In [ ]:
# special case 3: all 00 and 12 is missing -> use all previous day _12
import os
! python /content/drive/MyDrive/NOAA/grib2_files_test/gdal_calc.py \
    -A  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_12.grib2 \
    -B  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_13.grib2 \
    -C  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_14.grib2 \
    -D  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_15.grib2 \
    -E  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_16.grib2 \
    -F  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_17.grib2 \
    -G  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_18.grib2 \
    -H  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_19.grib2 \
    -I  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_20.grib2 \
    -J  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_21.grib2 \
    -K  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_22.grib2 \
    -L  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_23.grib2 \
    -M  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_24.grib2 \
    -N  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_25.grib2 \
    -O  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_26.grib2 \
    -P  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_27.grib2 \
    -Q  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_28.grib2 \
    -R  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_29.grib2 \
    -S  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_30.grib2 \
    -T  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_31.grib2 \
    -U  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_32.grib2 \
    -V  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_33.grib2 \
    -W  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_34.grib2 \
    -X  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201808/2018082512/postprd/nearsfc_smoke_35.grib2 \
    --outfile=/content/drive/MyDrive/HRRRsmoke_OS/averaged/20180826.tiff  \
    --calc="(A+B+C+D+E+F+G+H+I+J+K+L+M+N+O+P+Q+R+S+T+U+V+W+X)/24"

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 12.. 12.. 12.. 12.. 12.. 12.. 12.. 12.. 12.. 12.. 13.. 13.. 13.. 13.. 13.. 13.. 13.. 13.. 13.. 13.. 13.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 14.. 15.. 15.. 15.. 15.. 15.. 15.. 15.. 15.. 15.. 15.. 15.. 16.. 16.. 16.. 16.. 16.. 16.. 16.. 16.. 16.. 16.. 16.. 17.. 17.. 17.. 17.. 17.. 17.. 17.. 17.. 17.. 17.. 18.. 18.. 18.. 18.. 18.. 18.. 18.. 18.. 18.. 18.. 18.. 19.. 19.. 19.. 19.. 19.. 19.. 19.. 19.. 19.. 19.. 20.. 20.. 20.. 20.. 20.. 20.. 20.. 20.. 20.. 2

In [ ]:
# special case 4: all 00 and 12 is missing, previous 12 missing -> use all previous day _00
import os
! python /content/drive/MyDrive/NOAA/grib2_files_test/gdal_calc.py \
    -A  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_24.grib2 \
    -B  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_25.grib2 \
    -C  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_26.grib2 \
    -D  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_27.grib2 \
    -E  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_28.grib2 \
    -F  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_29.grib2 \
    -G  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_30.grib2 \
    -H  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_31.grib2 \
    -I  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_32.grib2 \
    -J  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_33.grib2 \
    -K  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_34.grib2 \
    -L  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_35.grib2 \
    -M  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_36.grib2 \
    -N  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_37.grib2 \
    -O  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_38.grib2 \
    -P  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_39.grib2 \
    -Q  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_40.grib2 \
    -R  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_41.grib2 \
    -S  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_42.grib2 \
    -T  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_43.grib2 \
    -U  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_44.grib2 \
    -V  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_45.grib2 \
    -W  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_46.grib2 \
    -X  /content/drive/MyDrive/HRRRsmoke_OS/grib2_files/201910/2019102100/postprd/nearsfc_smoke_47.grib2 \
    --outfile=/content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20191022.tiff  \
    --calc="(A+B+C+D+E+F+G+H+I+J+K+L+M+N+O+P+Q+R+S+T+U+V+W+X)/24"

In [ ]:
# Connect to Google Cloud Storage
from google.colab import auth
auth.authenticate_user()
project_id = 'amazing-blend-309705'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://noaa_os_2/


In [ ]:
# Transfer from drive to GCS
bucket_name = 'noaa_os_2'
!gsutil -m cp -r /content/drive/MyDrive/HRRRsmoke_OS/averaged2019/* gs://{bucket_name}/

Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190702.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190701.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190703.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190710.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190704.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190713.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190709.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190711.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/HRRRsmoke_OS/averaged2019/20190708.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive

In [ ]:
# Setup connection to Google Earth Engine 
import ee
# Trigger the authentication flow for gee
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Wsef0lspH2ZW_vnJHfkxZe1mupmNEYf-O3ISTePXlBY&tc=NJiMXyQ0yem6D6jecIgR2c1mXsZVAPYdEMZA7oynAJg&cc=xsPBb1o5qGNcYgdJ3i87IKOe8GnDAAvhkRY1EeO9xwU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvtCDh9rvBS6sKFasmhxslofUrk57xivxvyrfS_Pg8CLWEHjTgyO28Q

Successfully saved authorization token.


In [ ]:
# test GEE API to check connection: Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [ ]:
# Send files from GSC to GEE
!earthengine upload image --asset_id=users/sraby/noaa_daily_2018/20180731 gs://noaa_os_1/20180731.tiff


Started upload task with ID: SFM6TEFDDABIWSD45YWCF2AD


In [ ]:
# Send files from GSC to GEE
!earthengine upload image --asset_id=users/sraby/noaa_daily_2018/20180730 gs://noaa_os_1/20180730.tiff

Started upload task with ID: CWRKW6L32MEXJE7VVNFNPR44


In [ ]:
# Set start time on files (for time averaging in GEE)
!earthengine asset set --time_start 2018-07-01 users/sraby/noaa_daily_2018/20180701
# And that's a wrap! 